In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
# Warning is Gensim related

26-Jan-2017 18:30:36 DEBUG gensim.models.doc2vec: Fast version of gensim.models.doc2vec is being used
26-Jan-2017 18:30:36 INFO summa.preprocessing.cleaner: 'pattern' package not found; tag filters are not available for English
26-Jan-2017 18:30:36 INFO root: Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
26-Jan-2017 18:30:36 INFO root: Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
/home/tv/.virtualenvs/Topics/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/tv/.virtualenvs/Topics/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/tv/.virtualenvs/Topics/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signat

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

26-Jan-2017 18:30:36 INFO preprocessing: Creating document list from TXT files ...
26-Jan-2017 18:30:36 DEBUG preprocessing: 17 entries in document list.


['corpus_txt/Doyle_AScandalinBohemia.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt',
 'corpus_txt/Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt/Doyle_TheSignoftheFour.txt',
 'corpus_txt/Howard_GodsoftheNorth.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

26-Jan-2017 18:30:36 INFO preprocessing: Creating document list from CSV files ...
26-Jan-2017 18:30:36 DEBUG preprocessing: 16 entries in document list.


['corpus_csv/Doyle_AStudyinScarlet.txt.csv',
 'corpus_csv/Doyle_TheHoundoftheBaskervilles.txt.csv',
 'corpus_csv/Doyle_TheSignoftheFour.txt.csv',
 'corpus_csv/Howard_GodsoftheNorth.txt.csv',
 'corpus_csv/Howard_SchadowsinZamboula.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

26-Jan-2017 18:30:36 INFO preprocessing: Creating document labels ...
26-Jan-2017 18:30:36 DEBUG preprocessing: Document labels available


['corpus_txt/Doyle_AScandalinBohemia.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt',
 'corpus_txt/Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt/Doyle_TheSignoftheFour.txt',
 'corpus_txt/Howard_GodsoftheNorth.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[:1]

26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Doyle_AScandalinBohemia.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Doyle_AStudyinScarlet.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Doyle_TheHoundoftheBaskervilles.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Doyle_TheSignoftheFour.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_GodsoftheNorth.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_SchadowsinZamboula.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_ShadowsintheMoonlight.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_TheDevilinIron.txt ...
26-Jan-2017 18:30:37 DEBUG preprocessing: Accessing TXT document corpus_txt/Kipling_TheEndofthePassage.txt ...
26-Jan-2017 18:30

[['a',
  'scandal',
  'in',
  'bohemia',
  'a',
  'conan',
  'doyle',
  'i',
  'to',
  'sherlock',
  'holmes',
  'she',
  'is',
  'always',
  'the',
  'woman',
  'i',
  'have',
  'seldom',
  'heard',
  'him',
  'mention',
  'her',
  'under',
  'any',
  'other',
  'name',
  'in',
  'his',
  'eyes',
  'she',
  'eclipses',
  'and',
  'predominates',
  'the',
  'whole',
  'of',
  'her',
  'sex',
  'it',
  'was',
  'not',
  'that',
  'he',
  'felt',
  'any',
  'emotion',
  'akin',
  'to',
  'love',
  'for',
  'irene',
  'adler',
  'all',
  'emotions',
  'and',
  'that',
  'one',
  'particularly',
  'were',
  'abhorrent',
  'to',
  'his',
  'cold',
  'precise',
  'but',
  'admirably',
  'balanced',
  'mind',
  'he',
  'was',
  'i',
  'take',
  'it',
  'the',
  'most',
  'perfect',
  'reasoning',
  'and',
  'observing',
  'machine',
  'that',
  'the',
  'world',
  'has',
  'seen',
  'but',
  'as',
  'a',
  'lover',
  'he',
  'would',
  'have',
  'placed',
  'himself',
  'in',
  'a',
  'false'

#### Create Dictionaries

In [8]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [9]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [10]:
sparse_bow

0
doc_id token_id   
1      3         1
       8196      1
       5         1
       12295     1
       23212     1
       9684      3
       10242     1
       4110      1
       12305     5
       12306     2
       20500     6
       12310     4
       8222      1
       20511     1
       4130      9
       20516     5
       38        1
       8234      2
       20523     1
       8236      1
       8238      1
       16431     1
       20530     1
       16436     3
       20534     1
       14345     2
       4152      2
       4155      1
       16444     1
       16778     1
...             ..
16     4024      0
       16910     0
       22459     0
       1982      0
       18367     0
       14274     0
       9547      0
       15009     0
       18376     0
       21495     0
       4045      0
       338       0
       22485     0
       15353     0
       4056      0
       4431      0
       20444     0
       24541     0
       16352     0
       21685     0
       6115      0
       10213     0
       16361     0
       18413     0
       7507      0
       6142      0
       16912     0
       6139      0
       4094      0
       10239     0

[56956 rows x 1 columns]

#### Convert to Doc-Topic Matrix

In [11]:
'''import pandas as pd

doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

cols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]
#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]

set(sparse_bow.index.get_level_values("token_id"))

#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'''

'import pandas as pd\n\ndoc2id = {value : key for key, value in doc_ids.items()}\ntype2id = {value : key for key, value in id_types.items()}\n\ncols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]\n#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]\n\nset(sparse_bow.index.get_level_values("token_id"))\n\n#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'

#### Remove Features

In [12]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "en"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [13]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

26-Jan-2017 18:30:38 INFO preprocessing: Removing features ...
26-Jan-2017 18:30:38 DEBUG preprocessing: 672 features removed.


In [14]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

56956
51572


##### Create Feature Remove Lists

In [15]:
stopword_list = pre.find_stopwords(sparse_bow, id_types, 100)
len(stopword_list)

26-Jan-2017 18:30:38 INFO preprocessing: Finding stopwords ...
26-Jan-2017 18:30:38 DEBUG preprocessing: 100 stopwords found.


100

In [16]:
hapax_list = pre.find_hapax(sparse_bow, id_types)
len(hapax_list)

26-Jan-2017 18:30:38 INFO preprocessing: Find hapax legomena ...
26-Jan-2017 18:30:38 DEBUG preprocessing: 20647 hapax legomena found.


20647

##### Remove Features

In [17]:
feature_list = set(stopword_list).union(hapax_list)
clean_term_frequency = pre.remove_features(sparse_bow, id_types, feature_list)

26-Jan-2017 18:30:38 INFO preprocessing: Removing features ...
26-Jan-2017 18:30:38 DEBUG preprocessing: 20727 features removed.


In [18]:
len(clean_term_frequency)

5595

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [19]:
num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'w', encoding = "utf-8") as f:
    pass

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)

In [20]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [21]:
mm = MmCorpus("gb_plain.mm")

26-Jan-2017 18:30:38 INFO gensim.matutils: initializing corpus reader from gb_plain.mm
26-Jan-2017 18:30:38 INFO gensim.matutils: accepted corpus with 16 documents, 24552 features, 372528 non-zero entries


In [22]:
#in case you're only loading the corpus - build dict first

#import pickle

#pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [23]:
#id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [24]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [25]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [26]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

In [27]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id)

26-Jan-2017 18:30:38 INFO gensim.models.ldamodel: using symmetric alpha at 0.01
26-Jan-2017 18:30:38 INFO gensim.models.ldamodel: using symmetric eta at 4.0728220584042684e-05
26-Jan-2017 18:30:38 INFO gensim.models.ldamodel: using serial LDA version on this node
26-Jan-2017 18:31:04 INFO gensim.models.ldamodel: running online LDA training, 100 topics, 1 passes over the supplied corpus of 16 documents, updating model once every 16 documents, evaluating perplexity every 16 documents, iterating 50x with a convergence threshold of 0.001000
26-Jan-2017 18:31:04 WARNING gensim.models.ldamodel: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
26-Jan-2017 18:31:30 DEBUG gensim.models.ldamodel: bound: at document #0
26-Jan-2017 18:31:33 INFO gensim.models.ldamodel: -31.030 per-word bound, 2192109240.8 perplexity estimate based on a held-out corpus of 16 documents with 372528 words
26-Jan-2017 18:31:33 INFO gensim.models.ld

In [28]:
topic_nr_x = model.get_topic_terms(10)

[type2id[i[0]] for i in topic_nr_x]

['blacking',
 'earning',
 'room--had',
 'undisturbed',
 'indeed:—we',
 'criminal',
 'ununiformity',
 'menial',
 'primitive',
 'stares']

In [29]:
topics = model.show_topics(num_topics = 60)

In [30]:
topics

[(95,
  '0.033*"blacking" + 0.023*"earning" + 0.023*"room--had" + 0.021*"undisturbed" + 0.021*"criminal" + 0.017*"ununiformity" + 0.016*"indeed:—we" + 0.014*"bisher" + 0.011*"primitive" + 0.011*"snuffed"'),
 (69,
  '0.041*"blacking" + 0.027*"earning" + 0.025*"undisturbed" + 0.019*"room--had" + 0.018*"criminal" + 0.017*"stares" + 0.013*"indeed:—we" + 0.011*"primitive" + 0.011*"bisher" + 0.011*"snuffed"'),
 (97,
  '0.044*"blacking" + 0.036*"earning" + 0.022*"room--had" + 0.021*"undisturbed" + 0.021*"indeed:—we" + 0.016*"criminal" + 0.016*"ununiformity" + 0.015*"snuffed" + 0.013*"stares" + 0.013*"primitive"'),
 (18,
  '0.057*"blacking" + 0.029*"earning" + 0.024*"undisturbed" + 0.022*"criminal" + 0.022*"room--had" + 0.019*"stares" + 0.014*"ununiformity" + 0.013*"indeed:—we" + 0.012*"snuffed" + 0.011*"primitive"'),
 (3,
  '0.047*"blacking" + 0.029*"earning" + 0.025*"undisturbed" + 0.023*"criminal" + 0.022*"indeed:—we" + 0.017*"room--had" + 0.015*"snuffed" + 0.014*"ununiformity" + 0.013*"sta

## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [31]:
import os
basepath = os.path.abspath('.')

In [32]:
#path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
#path_to_mallet = "mallet"
path_to_corpus = os.path.join(basepath, 'corpus_txt')
assert os.path.exists(path_to_corpus)

# FIXME woher kommt der Ausgabepfad? Das muss doch in irgendeinen Parameter?
mal.create_mallet_binary(path_to_corpus)

#toDo absolute Pfade

26-Jan-2017 18:31:36 DEBUG mallet: mallet_output/malletBinary.mallet
26-Jan-2017 18:31:36 INFO mallet: Accessing Mallet ...


['mallet', 'import-dir', '--input', '/home/tv/git/Topics/corpus_txt', '--output', 'mallet_output/malletBinary.mallet', '--keep-sequence', '--remove-stopwords']


26-Jan-2017 18:31:36 DEBUG mallet: Mallet file available.


##### Mallet output erzeugen

In [33]:
outfolder = os.path.join(basepath, "mallet_output")
malletBinary = os.path.join(outfolder, "malletBinary.mallet")
mal.create_mallet_model(malletBinary, outfolder)

26-Jan-2017 18:31:36 DEBUG mallet: /home/tv/git/Topics/mallet_output
26-Jan-2017 18:31:36 INFO mallet: Accessing Mallet ...
26-Jan-2017 18:31:52 DEBUG mallet: Mallet file available.


## Visualisierung 

In [34]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

"lda_model = 'out_easy/corpus.lda'\ncorpus = 'out_easy/corpus.mm'\ndictionary = 'out_easy/corpus.dict'\ndoc_labels = 'out_easy/corpus_doclabels.txt'\ninteractive  = False\n\nvis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)"

#### Heatmap

In [35]:
#heatmap = visual.make_heatmap()

In [36]:
#visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [37]:
#vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [38]:
#vis.make_interactive()

In [39]:
#vis.save_interactive("./visualizations/interactive")

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)